# 1次元圧縮性Euler方程式
## 支配方程式
$\displaystyle \frac{\partial Q}{\partial t} + \frac{\partial F}{\partial x} = 0$

$\displaystyle e= \frac{1}{2}\rho u^2 + \frac{p}{\gamma-1}$

where  
$Q = \left[
\begin{matrix}
\rho \\ 
\rho u \\
e
\end{matrix}
\right]$

$F = \left[
\begin{matrix}
\rho u \\ 
\rho u^2 + p \\
(E + p)u
\end{matrix}
\right]$

$\rho$ is density of fluid, 
$u$ is flow velocity, 
$e$ is total energy per unit mass, and
$p$ is pressure.

## Flux jacobian
$\displaystyle \frac{\partial F}{\partial Q} = A = P\Lambda P^{-1}$

where 

$\displaystyle P=\left[
\begin{matrix}
1 & 1 & 1 \\ 
u-a & u & u+a \\
H-au & \frac{u^2}{2} & H+au
\end{matrix}
\right]$

$\displaystyle \Lambda = \left[
\begin{matrix}
u-a & 0 & 0 \\ 
0 & u & 0 \\
0 & 0 & u+a
\end{matrix}
\right]$

$\displaystyle P^{-1}=\left[
\begin{matrix}
\frac{1}{2}\left( \alpha_1 + \frac{u}{a} \right) & -\frac{1}{2}\left( \alpha_2 u + \frac{1}{a} \right) & \frac{\alpha_2}{2} \\ 
1-\alpha_1 & \alpha_2u & -\alpha_2 \\
\frac{1}{2}\left( \alpha_1 - \frac{u}{a} \right) & -\frac{1}{2}\left( \alpha_2 u - \frac{1}{a} \right) & \frac{\alpha_2}{2} \\ 
\end{matrix}
\right]$

$\displaystyle \alpha_1 = 
\frac{\left( \gamma - 1 \right) u^2}
{2a^2}$

$\displaystyle \alpha_2 = 
\frac{\left( \gamma - 1 \right)}
{a^2}$

$H$ is total enthalpy, $T$ is temperature, and $a$ is speed of sound.

$\displaystyle H = \rho c_p T = \frac{\gamma}{\gamma - 1} \rho T = \frac{\gamma}{\gamma - 1}p$  
$\displaystyle T = \frac{p}{\rho}$  
$\displaystyle a = \sqrt{ \gamma \frac{p}{\rho} }$


$\gamma = 1.4$ is heat capacity ratio.

## descritization
$\displaystyle \int_\Omega \left( \frac{\partial Q_i}{\partial t} + \frac{\partial F_i}{\partial x} \right) d\Omega= 0$

$\displaystyle \frac{\partial}{\partial t} \int_\Omega Q_i d\Omega + \int_{ \partial \Omega } F_i \vec{n} d\left( \partial \Omega \right)= 0$

$\displaystyle \frac{\partial}{\partial t} \bar{Q} \delta \Omega = - \sum_{ j \in \partial \Omega  } F_{ij} \vec{n}_j \delta (\partial \Omega)_j$

where  
$\displaystyle F_{ij} = \frac{1}{2} \left( F_i + F_j \right) - \frac{1}{2} A^{roe} \left( Q_j - Q_i \right)$

$A^{roe} = P\Lambda P^{-1}$  
Note that $P$, $\Lambda$ and $P^{-1}$ are calculated from the variable of roe average.

### Roe average
$\rho_{roe} = \sqrt{\rho_i \rho_j}$  
$\displaystyle u_{roe} = \frac{\sqrt{\rho_i} u_i + \sqrt{\rho_j} u_j}{\sqrt{\rho_i} + \sqrt{\rho_j} }$  
$\displaystyle H_{roe} = \frac{\sqrt{\rho_i} H_i + \sqrt{\rho_j} H_j}{\sqrt{\rho_i} + \sqrt{\rho_j} }$  
$\displaystyle a_{roe} = \sqrt{ \left( \gamma - 1 \right) \left( H_{roe} - \frac{u_{roe}^2}{2} \right) }$  

where H is total enthalpy per unit volume as below.  
$\displaystyle H = \frac{\left( e + p\right)}{\rho} = \frac{1}{\rho} \left( \frac{1}{2}\rho u^2 + \frac{p}{\gamma-1} + p  \right) = \frac{1}{2}u^{2} + \frac{\gamma}{\gamma-1}\frac{p}{\rho}$

In [2]:
# constant
L=3 # [m]
U=5 # [m/s]
P=101325 # [Pa]
Rho = 1.293 # [m^3/kg]
T=298 # [K]

g = 1.4
g1 = g - 1
g3 = g - 3

non_L = 1
non_U = 1
non_A = 1
non_Rho = 1
non_P = non_Rho / g * non_A**2
non_T = non_P / non_Rho
non_e = 1/2 * non_Rho * non_U**2 + non_P / g1
print(non_P)
print(non_e)
def e2p(Q):
    rho = Q[:, 0]
    u = Q[:, 1] / Q[:, 0]
    e = Q[:, 2]
    p = g1 * ( e - 1/2 * rho * u**2 )
    return p

def e2T(Q):
    p = e2p(Q)
    rho = Q[:, 0]
    T = p / rho
    return T

0.7142857142857143
2.2857142857142865


In [3]:

from math import sqrt
import numpy as np

def q2b(q):
    rho = q[0]
    u = q[1] / q[0]
    e = q[2] 
    p = g1 * ( e - (1/2 * rho * u**2) )
    return rho, u, e, p

def q2f(q):
    (rho, u, e, p) = q2b(q)
    F = np.zeros(3)
    F[0] = rho * u
    F[1] = rho * u**2 + p
    F[2] = (e + p) * u
    #F[0] = rho * u
    #F[1] = -g3/2 * rho * u**2 + g1*e
    #F[2] = -g1/2 * rho*u**3 + g*u*e
    return F

def entropy_fix(a, epsilon=0.15):
    abs_a = abs(a)
    if ( abs_a < epsilon ):
        mod_a = 1/2* (abs_a**2/epsilon + epsilon)
    else:
        mod_a = abs_a
    return mod_a

def roe_fds(qi, qj):
    ( rho_i, u_i, e_i, p_i ) = q2b(qi)
    ( rho_j, u_j, e_j, p_j ) = q2b(qj)
    H_i = ( g / g1 ) * p_i / rho_i + u_i**2 / 2
    H_j = ( g / g1 ) * p_j / rho_j + u_j**2 / 2
    
    rho = sqrt(rho_i * rho_j)
    u = ( sqrt(rho_i) * u_i + sqrt(rho_j) * u_j ) / (sqrt(rho_i) + sqrt(rho_j) )
    H = ( sqrt(rho_i) * H_i + sqrt(rho_j) * H_j ) / (sqrt(rho_i) + sqrt(rho_j) )
    a_sq = g1 * ( H - u**2 /2 )
    if a_sq < 0:
        print(qi, qj)
        print(rho, u, H, a_sq)
        raise ValueError
    # a = sqrt( g1 * ( H - u**2/2 ) )
    a = sqrt(a_sq)
    if ( rho_i != rho_j ):
        pass
        # print(rho_i, u_i, e_i, p_i)
        # print(rho_j, u_j, e_j, p_j)
        # print(H_i, H_j, sqrt(rho_i)*H_i, sqrt(rho_j)*H_j, sqrt(rho_i) + sqrt(rho_j), H )
        # print(rho, g1*H/a_sq, a_sq)
    
    P = np.zeros((3,3))
    invP = np.zeros((3,3))
    L = np.zeros((3,3))
    
    a1 = ( g1 * u**2 ) / (2 * a**2)
    a2 = g1 / a**2
    
    P[0, 0] = 1
    P[0, 1] = 1
    P[0, 2] = 1
    P[1, 0] = u - a
    P[1, 1] = u
    P[1, 2] = u + a
    P[2, 0] = H - a * u
    P[2, 1] = u**2 / 2
    P[2, 2] = H + a * u

    L[0, 0] = entropy_fix(u - a)
    L[1, 1] = entropy_fix(u)
    L[2, 2] = entropy_fix(u + a) 
    
    invP[0, 0] = 1/2 * (a1 + u/a)
    invP[0, 1] = -1/2 * (a2*u + 1/a)
    invP[0, 2] = a2 / 2
    invP[1, 0] = 1 - a1
    invP[1, 1] = a2 * u
    invP[1, 2] = -a2
    invP[2, 0] = 1/2*(a1 - u/a)
    invP[2, 1] = -1/2 * (a2*u - 1/a)
    invP[2, 2] = a2 / 2

    dq = qj - qi
    dF = np.dot(P, np.dot(L, np.dot(invP, dq)))
    
    F_i = q2f(qi)
    F_j = q2f(qj)
    #print(qi, qj)
    #print(F_i, F_j)
    roeF = 1/2 * (F_i + F_j) - 1/2*dF
    return roeF  

#### SLAU2 algorithm
$\displaystyle \bar{\rho} = \frac{\rho_L + \rho_R}{2}$  
$\displaystyle \bar{c} = \frac{c_L + c_R}{2}$  
$\displaystyle \hat{M} = min\left( 1, \frac{1}{\bar{c}} \sqrt{ \frac{u_L^2 + u_R^2}{2}}\right)$  

$\displaystyle M_L = \frac{V_{nL}}{\bar{c}} = \frac{\vec{u_L} \bullet \vec{n_L}}{\bar{c}}$  
$\displaystyle M_R = \frac{V_{nR}}{\bar{c}} = \frac{\vec{u_R} \bullet \vec{n_R}}{\bar{c}}$  
$\displaystyle \left| \bar{V_n} \right| = \frac{\rho_L \left| V_{nL} \right| + \rho_R \left| V_{nR} \right| }{\rho_L + \rho_R}$  

$\displaystyle P_L^+ = ...$  
$\displaystyle P_R^- = ...$  
$\displaystyle P^{\pm}|_{\alpha} = \left\{ \begin{array}{ll} \frac{1}{2} \left( 1 \pm sign\left( M \right) \right) & \left( \left| M \right| \ge 1 \right) \\ \frac{1}{4} \left( M \pm 1 \right)^2 \left( 2 \mp M \right) \pm \alpha M \left( M^2 - 1\right)^2 & \left( \left|M\right| \lt 1 \right) \end{array} \right.$  
$\displaystyle \tilde{p} = \frac{p_L + p_R}{2} + \frac{P^+|_{\alpha=0} - P^-|_{\alpha=0}}{2} \left( p_{L} - p_{R} \right) + \gamma_{HR} \sqrt{\frac{u_L^2+u_R^2}{2}} \left( P^+|_{\alpha=0} + P^-|_{\alpha=0} - 1 \right) \bar{\rho} \bar{c}$  

$\displaystyle g = - \left[ max\left\{ min \left( M_L, 0 \right), -1 \right\} \bullet min\left\{ max\left( M_R, 0 \right), 1 \right\} \right]$  
$\displaystyle \chi = \left( 1 - \hat{M}^2 \right) $  

$\displaystyle \left| \bar{V_n} \right|^+ = (1-g)\left| \bar{V_n} \right| + g\left| V_{nL} \right|$  
$\displaystyle \left| \bar{V_n} \right|^- = (1-g)\left| \bar{V_n} \right| + g\left| V_{nR} \right|$  
$\displaystyle \dot{m} = \frac{1}{2} \left\{ \rho_L \left( V_{nL} + \left| \bar{V_n} \right|^+ \right) + \rho_R \left( V_{nR} + \left| \bar{V_n} \right|^- \right)\right\} - \frac{\chi}{\bar{c}} \left( p_R - p_L \right)$  
$\displaystyle E=\frac{\dot{m} + \left| \dot{m} \right|}{2}\psi^ + \frac{\dot{m} - \left| \dot{m} \right|}{2}\psi^- +\tilde{p}\vec{N} $  

In [4]:
def slau2(qi, qj):
    rho_L, u_L, e_L, p_L = q2b(qi)
    rho_R, u_R, e_R, p_R = q2b(qj)
    n_L = 1
    n_R = 1
    H_L = ( g / g1 ) * p_L / rho_L + u_L**2 / 2
    H_R = ( g / g1 ) * p_R / rho_R + u_R**2 / 2
    
    c_L = sqrt(g * p_L / rho_L)
    c_R = sqrt(g * p_R / rho_R)
    
    bar_rho = (rho_L + rho_R) / 2
    bar_c = (c_L + c_R) / 2
    
    hat_M = min(1, 1/bar_c * sqrt((u_L**2 + u_R**2)/2))
    VnL = u_L * n_L
    VnR = u_R * n_R
    
    M_L = VnL / bar_c
    M_R = VnR / bar_c
    
    alpha = 0
    if ( abs(M_L) > 1 ):
        Pplus_L = 1/2 * (1 + np.sign(M_L) )
    else:
        Pplus_L = 1/4 * (M_L + 1)**2 * (2 - M_L) + alpha * M_L * (M_L**2 - 1)**2
    
    if ( abs(M_R) > 1 ):
        Pminus_R = 1/2 * (1 + np.sign(M_R) )
    else:
        Pminus_R = 1/4 * (M_R - 1)**2 * (2 + M_R) - alpha * M_R * (M_R**2 - 1)**2
    
    # この値の計算はi-1, j+1が必要なため割愛
    # gamma_w = 1 - np.sign(min(phi_1, phi_2))
    gamma_w = 1    
    gamma_HR = max(0.2 , gamma_w)
    
    tilde_p = (p_L + p_R)/2 + (Pplus_L + Pminus_R)/2 * (p_L - p_R) + gamma_HR * sqrt( (u_L**2 + u_R**2 )/2 ) * (Pplus_L + Pminus_R - 1) * bar_rho * bar_c
    g_slau = -( max( min(M_L, 0), -1 ) * min( max(M_R, 0), 1) )
    chi = 1 - hat_M**2
    
    abs_Vn = ( rho_L * abs(VnL) + rho_R * abs(VnR) ) / ( rho_L + rho_R )
    Vn_plus = (1-g_slau)*abs_Vn + g_slau * abs(VnL)
    Vn_minus = (1-g_slau)*abs_Vn + g_slau * abs(VnR)
    
    m_dot = 1/2 * ( rho_L * (VnL + Vn_plus) + rho_R * (VnR - Vn_minus)) - chi/bar_c * (p_R - p_L)
    
    psi_L = np.array([1, u_L, H_L])
    psi_R = np.array([1, u_R, H_R])
    N = np.array([0, 1, 0])
    
    F = (m_dot + abs(m_dot))/2 * psi_L + (m_dot - abs(m_dot))/2 * psi_R + tilde_p * N

    return F  


In [202]:
def Q2B(Q):
    B = np.zeros_like(Q)
    Rho = Q[:, 0]
    U = Q[:, 1] / Q[:, 0]
    E = Q[:, 2] 
    P = g1 * ( E - (Rho * U**2 / 2) )
    B[:, 0] = Rho
    B[:, 1] = U
    B[:, 2] = P
    return B

def B2Q(B):
    Q = np.zeros_like(B)
    Q[:, 0] = B[:, 0]
    Q[:, 1] = B[:, 0] * B[:, 1]
    Q[:, 2] = 1/2*B[:, 0] * B[:, 1]**2 + B[:, 2] / g1
    return Q

def muscl(Q, qB):
    def minmod(p, q, b):
        # print(np.sign(p), abs(p), q*b, min(abs(p), np.sign(p)*q*b))
        return np.sign(p) * max(0, min(abs(p), np.sign(p)*q*b))
    
    B = Q2B(Q)
    
    # 1 と 2 の間を1に格納
    BL = np.zeros((n+1, 3))
    BR = np.zeros((n+1, 3))
    
    qq0 = np.zeros(3)
    qq1 = np.zeros(3)
    qq2 = np.zeros(3)
    k_muscl = 1/3
    b_muscl = (3-k_muscl) / (1-k_muscl)
    for i in range(n):
        if ( i == 0 ):
            qq0[0], qq0[1], _, qq0[2] = q2b(qB[:, 0])
        else:
            qq0 = B[i-1]
            
        qq1 = B[i]
        
        if ( i == n-1 ):
            qq2[0], qq2[1], _, qq2[2] = q2b(qB[:, 1])
        else:
            qq2 = B[i+1]
        
        for k in range(3):
            dplus_i = qq2[k] - qq1[k]
            dminus_i = qq1[k] - qq0[k]
            
            # i + 1/2
            BL[i+1, k] = qq1[k] + 1/4*(1-k_muscl)*minmod(dminus_i, dplus_i, b_muscl) + 1/4*(1+k_muscl)*minmod(dplus_i, dminus_i, b_muscl)
            # BL[i+1, k] = qq1[k] + 1/4*(1-k_muscl)*dminus_i + 1/4*(1+k_muscl)*dplus_i
            
            # i - 1/2
            BR[i, k] = qq1[k] - 1/4*(1-k_muscl)*minmod(dplus_i, dminus_i, b_muscl) - 1/4*(1+k_muscl)*minmod(dminus_i, dplus_i, b_muscl)
            # BR[i, k] = qq1[k] - 1/4*(1-k_muscl)*dplus_i - 1/4*(1+k_muscl)*dminus_i
        
    # BL[0, :] = q2b(qB[:, 0])
    # BR[n, :] = qq2[k]
            
    QL = B2Q(BL)
    QR = B2Q(BR)
    QL[0, :] = qB[:, 0]
    QR[n, :] = qB[:, 1]
    # print(QL)
    # print(QR)
    # print(sqrt(-1))
    return QL, QR



In [6]:
def fvs(qi, qj):   
    R, R_inv, L, L_abs = A_pm(qi)
    Ap = np.dot(np.dot(R, L+L_abs), R_inv)
    
    R, R_inv, L, L_abs = A_pm(qj)
    Am = np.dot(np.dot(R, L-L_abs), R_inv)
    
    F_plus = 1/2 * ( np.dot(Ap, qi) + np.dot(Am, qj))
    return F_plus

def A_pm(q):
    rho, u, e, p = q2b(q)
    H = (e + p) / rho
    c = sqrt(g1 * (H - u**2 / 2))
    b2 = g1 / c**2
    b1 = (b2 * u**2) / 2
    
    R = np.array([[1, 1, 1], [u-c, u, u+c], [H-u*c, u**2 / 2, H+u*c]])
    R_inv = np.array([[(b1+u/c)/2, (-b2*u-1/c)/2, b2/2],
                      [1-b1, b2*u, -b2], 
                      [(b1-u/c)/2, (-b2*u+1/c) / 2, b2/2]])
    
    L = np.array([[u-c, 0, 0],[0, u, 0],[0, 0, u+c]])
    L_abs = np.array([[abs(u-c), 0, 0],[0, abs(u), 0],[0, 0, abs(u+c)]])
    return R, R_inv, L, L_abs
    


In [199]:
def flux(Q, qB):
    F = np.zeros((n+1, 3))
    if use_muscl:
        QL, QR = muscl(Q, qB)
        # sqrt(-1)
        
    for j in range(n+1):
        i = j
        if ( j == 0 ):
            qi = qB[:, 0]
        else:
            if use_muscl:
                qi = QL[i] # L
            else:
                qi = Q[j-1, :]

        if ( j == n ):
            qj = qB[:, 1]
        else:
            if use_muscl:
                qj = QR[i] # R
            else:
                qj = Q[j  , :]
        
        F[j, :] = slau2(qi, qj)
        # F[j, :] = fvs(qi, qj)
        # if ( qi[0] != qj[0] ):
            # print(j, qi, qj, F[j, :])
        # F[j, :] = roe_fds(qi, qj)
    # sqrt(-1)
    return F
    

In [175]:
Tmax = 3 / (L/U) # [-] non-dimentional time
dt = 0.001
n = 100
dx = non_L / n
x_c = np.zeros(n)
x_e = np.zeros(n+1)
for j in range(n+1):
    x_e[j] = j * dx
    
for i in range(n):
    x_c[i] = (i + 1 / 2 ) * dx
    
# initial condition
Q = np.zeros((n, 3), np.float64)
Q[:, 0] = non_Rho
Q[:, 1] = non_Rho * 0
Q[:, 2] = 1/2*non_Rho*0**2 + ( non_P ) / g1

for i in range(n):
    if ( x_c[i] < 0.5 ):
        Q[i, 0] = 1/8*non_Rho
        Q[i, 2] = 1/2*(1/8*non_Rho)*0**2 + ( non_P / 10 ) / g1


In [120]:
def test():
    qi = np.array([1, 0, 1.78571429])
    qj = np.array([0.125, 0, 0.17857143])
    F1 = slau2(qi, qj)
    F2 = slau2(qj, qi)
    # F1 = fvs(qi, qj)
    # F2 = fvs(qj, qi)
    print(F1)
    print(F2)
    
test()
    

[0.67868234 0.71428572 1.69670586]
[-0.67868234  0.07142857 -1.69670586]


In [10]:
import matplotlib.pyplot as plt
# plt.scatter(x_c, Q[:, 0])
# plt.scatter(x_c, Q[:, 1])
# plt.scatter(x_c, g1*(Q[:, 2] - 1/2*Q[:, 1]**2/Q[:, 0]))

In [11]:
def bc_refrect(q):
    rho, u, e, p = q2b(q)
    qB = np.zeros(3)
    qB[0] = rho
    qB[1] = - rho * u
    qB[2] = e
    return qB

In [12]:
def calc_CFL(Q, dx, dt):
    CFL = 0
    for i in range(n):
        qi = Q[i, :]
        ( rho, u, e, p ) = q2b(qi)
        a = sqrt(g * p / rho)
        CFL_i = max(abs(u), abs(u-a), abs(u+a)) * dt / dx
        CFL = max(CFL, CFL_i)
    return CFL

### explicit euler
$\displaystyle Q_{i}^{n+1} = Q_{i} - \frac{dt}{\delta \Omega} \sum_{ j \in \partial \Omega  } F_{ij} \vec{n}_j \delta (\partial \Omega)_j$

In [196]:
plot_dt = 100
Nmax = int(0.15 / dt)#int(Tmax / dt)
Pmax = int(Nmax / plot_dt )
use_muscl = True
def explicit_euler(Q):
    qB = np.zeros((3, 2))
    Q_store = np.zeros((n, 3, Pmax+1))
    for k in range(Nmax):
        # print(k, calc_CFL(Q, dx, dt))
        
        # set boundary 
        qB[:, 0] = bc_refrect(Q[0, :])
        qB[:, 1] = bc_refrect(Q[n-1, :])
        
        F = flux(Q, qB)
        #print(F)
        Qn1 = np.zeros((n, 3))
        for i in range(n):
            m1 = i
            p1 = i + 1
            # print(k, i)
            Qn1[i, :] = Q[i, :] - dt / dx * (F[p1, :] - F[m1, :])
            
        if ( k % plot_dt == 0 ):
            print(k)
            m = int(k / plot_dt)
            Q_store[:, 0, m] = Q[:, 0]
            Q_store[:, 1, m] = Q[:, 1]
            Q_store[:, 2, m] = Q[:, 2]
            
        Q = Qn1
    return Q_store
    
Qsr = explicit_euler(Q)
    

0
100


In [203]:
plot_dt = 10
Nmax = 300#int(0.15 / dt)#int(Tmax / dt)
Pmax = int(Nmax / plot_dt )
use_muscl = True#False#True
def rk2(Q):
    qB = np.zeros((3, 2))
    Q_store = np.zeros((n, 3, Pmax))
    for k in range(Nmax):
        # print(k, calc_CFL(Q, dx, dt))
        
        # set boundary 
        qB[:, 0] = bc_refrect(Q[0, :])
        qB[:, 1] = bc_refrect(Q[n-1, :])
        
        F = flux(Q, qB)
        #print(F)
        Qn1 = np.zeros((n, 3))
        for i in range(n):
            m1 = i
            p1 = i + 1
            # print(k, i)
            Qn1[i, :] = Q[i, :] - dt / dx * (F[p1, :] - F[m1, :])
        
        # F_0 = F
        # F_1 = flux(Qn1, qB)
        # F = (F_0 + F_1)/2
        F = flux(Qn1, qB)
        
        # Qn1 = np.zeros((n, 3))
        for i in range(n):
            m1 = i
            p1 = i + 1
            # print(k, i)
            Qn1[i, :] = 1/2* ( Q[i, :] + Qn1[i, :] - dt / dx * (F[p1, :] - F[m1, :]) )
            
        if ( k % plot_dt == 0 ):
            print(k)
            m = int(k / plot_dt)
            Q_store[:, 0, m] = Q[:, 0]
            Q_store[:, 1, m] = Q[:, 1]
            Q_store[:, 2, m] = Q[:, 2]
            
        Q = Qn1
    return Q_store
    
Qsr = rk2(Q)
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290


In [204]:
rho_plt = Qsr[:, 0, :]
u_plt = Qsr[:, 1, :] / Qsr[:, 0, :] 
p_plt = ( Qsr[:, 2, :] - 1/2 * rho_plt * u_plt**2 ) * g1

from matplotlib.animation import ArtistAnimation
from IPython import display 

fig = plt.figure(figsize=(5,10))
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)
plt.xlim(0, 1)
# ax1.set_ylim(0, int(np.max(rho_plt)+0.5))
# ax2.set_ylim(-1, 1)
# ax3.set_ylim(0, 2)


frames = []
x = x_c
y1 = rho_plt
y2 = u_plt
y3 = p_plt
num_frames = rho_plt.shape[1]
for i in range(num_frames):
    
    frame1 = ax1.plot(x, y1[:, i], label='Density', color='blue')
    frame2 = ax2.plot(x, y2[:, i], label='Velocity', color='orange')
    frame3 = ax3.plot(x, y3[:, i], label='Pressure', color='green')
    
    if ( i == 0 ):
        ax1.legend()
        ax2.legend()
        ax3.legend()
    frames.append(frame1 + frame2 + frame3)
    
# 描画
ani = ArtistAnimation(fig, frames, interval=100)
html = display.HTML(ani.to_jshtml())
display.display(html)
plt.close()